In [2]:
import sagemaker
import boto3
import sys
import os
import glob
import re
import subprocess
import numpy as np
from IPython.display import HTML
import time
from time import gmtime, strftime
import nes_py



COMMON_PATH = "amazon-sagemaker-examples/reinforcement_learning/common"
sys.path.append(COMMON_PATH)
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework

In [3]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

S3 bucket path: s3://sagemaker-eu-central-1-131765425223/


In [4]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

S3 bucket path: s3://sagemaker-eu-central-1-131765425223/


In [11]:
job_name_prefix = 'rl-super-mario'

In [5]:
# run in local mode?
local_mode = True

if local_mode:
    instance_type = 'local'
else:
    instance_type = "ml.m4.4xlarge"

In [6]:
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()

print("Using IAM role arn: {}".format(role))

Using IAM role arn: arn:aws:iam::131765425223:role/service-role/AmazonSageMaker-ExecutionRole-20200812T110893


In [8]:
!pygmentize src/mario-dqn.py

from rl_coach.agents.dqn_agent import DQNAgentParameters
from rl_coach.base_parameters import VisualizationParameters, PresetValidationParameters, DistributedCoachSynchronizationType
from rl_coach.core_types import TrainingSteps, EnvironmentEpisodes, EnvironmentSteps
from rl_coach.environments.gym_environment import GymEnvironmentParameters, GymVectorEnvironment
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import ScheduleParameters
from rl_coach.memories.memory import MemoryGranularity
from rl_coach.schedules import LinearSchedule

####################
# Graph Scheduling #
####################

schedule_params = ScheduleParameters()
schedule_params.improve_steps = TrainingSteps(10000000000)
schedule_params.steps_between_evaluation_periods = EnvironmentEpisodes(10)
schedule_params.evaluation_steps = EnvironmentEpisodes(1)
schedule_params.heatup_steps = EnvironmentSteps(1000)

#########
# Agent #
#########
agent

In [9]:
!pygmentize src/train-coach.py

from sagemaker_rl.coach_launcher import SageMakerCoachPresetLauncher

class MyLauncher(SageMakerCoachPresetLauncher):

    def default_preset_name(self):
        """This points to a .py file that configures everything about the RL job.
        It can be overridden at runtime by specifying the RLCOACH_PRESET hyperparameter.
        """
        return 'mario-dqn'
    
if __name__ == '__main__':
    MyLauncher.train_main()


In [12]:
estimator = RLEstimator(entry_point="train-coach.py",
                        source_dir='src',
                        toolkit=RLToolkit.COACH,
                        toolkit_version='0.11.0',
                        dependencies=[COMMON_PATH +"/sagemaker_rl"],
                        framework=RLFramework.TENSORFLOW,
                        role=role,
                        train_instance_type=instance_type,
                        train_instance_count=1,
                        output_path=s3_output_path,
                        base_job_name=job_name_prefix
                    )

estimator.fit(wait=local_mode)

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Creating tmpa31ifcca_algo-1-yxvlr_1 ... 
Attaching to tmpa31ifcca_algo-1-yxvlr_12mdone
algo-1-yxvlr_1  | 2020-11-17 15:30:14,524 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-yxvlr_1  | 2020-11-17 15:30:14,529 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-yxvlr_1  | 2020-11-17 15:30:14,722 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-yxvlr_1  | 2020-11-17 15:30:14,737 sagemaker-containers INFO     Invoking user script
algo-1-yxvlr_1  | 
algo-1-yxvlr_1  | Training Env:
algo-1-yxvlr_1  | 
algo-1-yxvlr_1  | {
algo-1-yxvlr_1  |     "additional_framework_parameters": {
algo-1-yxvlr_1  |         "sagemaker_estimator": "RLEstimator"
algo-1-yxvlr_1  |     },
algo-1-yxvlr_1  |     "channel_input_dirs": {},
algo-1-yxvlr_1  |     "current_host": "algo-1-yxvlr",
algo-1-yxvlr_1  |     "framework_module": "sagemaker_tensorflow_container.training:main",
algo-1-yxvlr_1  |     

Failed to delete: /tmp/tmpa31ifcca/algo-1-yxvlr Please remove it manually.


RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpa31ifcca/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1